## Importing libraries and dataset

In [332]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns

In [333]:
customer_df_cleaned = customer_df_cleaned.loc[:, ~customer_df_cleaned.columns.str.contains('^Unnamed')]


In [334]:
customer_df_cleaned

,state,customer_lifetime_value,response,coverage,education,employmentstatus,gender,income,location_code,marital_status,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,contracts_started_in_2011
0,Washington,2763.519279,No,Basic,Bachelor,Employed,F,56274.0,Suburban,Married,...,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize,02-24
1,Arizona,6979.535903,No,Extended,Bachelor,Unemployed,F,0.0,Suburban,Single,...,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize,01-31
2,Nevada,12887.431650,No,Premium,Bachelor,Employed,F,48767.0,Suburban,Married,...,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize,02-19
3,California,7645.861827,No,Basic,Bachelor,Unemployed,M,0.0,Suburban,Married,...,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize,01-20
4,Washington,2813.692575,No,Basic,Bachelor,Employed,M,43836.0,Rural,Single,...,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize,02-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8692,California,23405.987980,No,Basic,Bachelor,Employed,M,71941.0,Urban,Married,...,0,2,Personal Auto,Personal L1,Offer2,Web,198.234764,Four-Door Car,Medsize,02-10
8693,California,3096.511217,Yes,Extended,College,Employed,F,21604.0,Suburban,Divorced,...,0,1,Corporate Auto,Corporate L3,Offer1,Branch,379.200000,Four-Door Car,Medsize,02-12
8694,California,8163.890428,No,Extended,Bachelor,Unemployed,M,0.0,Suburban,Single,...,3,2,Corporate Auto,Corporate L2,Offer1,Branch,790.784983,Four-Door Car,Medsize,02-06
8695,California,7524.442436,No,Extended,College,Employed,M,21941.0,Suburban,Married,...,0,3,Personal Auto,Personal L2,Offer3,Branch,691.200000,Four-Door Car,Large,02-03


## Normalize the continuous variables. You can use any one method you want

In [335]:
# 'Total Claim Amount' is going to be our target,so we dont scale this column
continious = customer_df_cleaned[['customer_lifetime_value','income']]
discrete = customer_df_cleaned[['monthly_premium_auto','months_since_last_claim','months_since_policy_inception',
               'number_of_open_complaints','number_of_policies']]

In [336]:
from sklearn.preprocessing import StandardScaler
transformer = StandardScaler().fit(continious)
continious_scaled = pd.DataFrame(transformer.transform(continious))

In [337]:
continious_scaled

,0,1
0,-0.961822,0.611236
1,0.015265,-1.234670
2,1.384454,0.364991
3,0.169690,-1.234670
4,-0.950194,0.203244
...,...,...
8692,3.822192,1.125147
8693,-0.884649,-0.526013
8694,0.289746,-1.234670
8695,0.141550,-0.514959


In [338]:
continious_scaled = continious_scaled.set_axis(['customer_lifetime_value', 
                                                        'income'], axis=1, inplace=False)

In [339]:
continious_scaled.head()

,customer_lifetime_value,income
0,-0.961822,0.611236
1,0.015265,-1.234670
2,1.384454,0.364991
3,0.169690,-1.234670
4,-0.950194,0.203244


## 17.Encode the categorical variables

In [340]:
from sklearn.preprocessing import OrdinalEncoder

oe = OrdinalEncoder()
ordinal_cols = ['education', 'coverage','employmentstatus','vehicle_size','location_code']
customer_df_cleaned[ordinal_cols] = oe.fit_transform(customer_df_cleaned[ordinal_cols]).astype(int)

In [341]:
ordinal_encoded = customer_df_cleaned[ordinal_cols].astype(int)
ordinal_encoded

,education,coverage,employmentstatus,vehicle_size,location_code
0,0,0,1,1,1
1,0,1,4,1,1
2,0,2,1,1,1
3,0,0,4,1,1
4,0,0,1,1,0
...,...,...,...,...,...
8692,0,0,1,1,2
8693,1,1,1,1,1
8694,0,1,4,1,1
8695,1,1,1,0,1


## One Hot Encoding

In [342]:
onehot_cols = customer_df_cleaned[['state','marital_status','policy_type','sales_channel','policy','vehicle_class',
                                   'renew_offer_type']]

In [343]:
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
enc = preprocessing.OneHotEncoder()
enc.fit(onehot_cols)
onehotlabels = enc.transform(onehot_cols).toarray()

In [344]:
pd.DataFrame(onehotlabels)

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8692,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8693,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8694,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8695,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [345]:
enc.categories_

[array(['Arizona', 'California', 'Nevada', 'Oregon', 'Washington'],
       dtype=object),
 array(['Divorced', 'Married', 'Single'], dtype=object),
 array(['Corporate Auto', 'Personal Auto', 'Special Auto'], dtype=object),
 array(['Agent', 'Branch', 'Call Center', 'Web'], dtype=object),
 array(['Corporate L1', 'Corporate L2', 'Corporate L3', 'Personal L1',
        'Personal L2', 'Personal L3', 'Special L1', 'Special L2',
        'Special L3'], dtype=object),
 array(['Four-Door Car', 'Luxury Car', 'Luxury SUV', 'SUV', 'Sports Car',
        'Two-Door Car'], dtype=object),
 array(['Offer1', 'Offer2', 'Offer3', 'Offer4'], dtype=object)]

In [346]:
#This syntax is for getting back our dataset columns in encoded dataframe

from functools import reduce
one_hot_cols_encoded = reduce(lambda x, y: list(x) + list(y), enc.categories_)

In [347]:
one_hot_cols_encoded

['Arizona',
 'California',
 'Nevada',
 'Oregon',
 'Washington',
 'Divorced',
 'Married',
 'Single',
 'Corporate Auto',
 'Personal Auto',
 'Special Auto',
 'Agent',
 'Branch',
 'Call Center',
 'Web',
 'Corporate L1',
 'Corporate L2',
 'Corporate L3',
 'Personal L1',
 'Personal L2',
 'Personal L3',
 'Special L1',
 'Special L2',
 'Special L3',
 'Four-Door Car',
 'Luxury Car',
 'Luxury SUV',
 'SUV',
 'Sports Car',
 'Two-Door Car',
 'Offer1',
 'Offer2',
 'Offer3',
 'Offer4']

In [348]:
onehotlabels = pd.DataFrame(onehotlabels, columns = one_hot_cols_encoded)
onehotlabels

,Arizona,California,Nevada,Oregon,Washington,Divorced,Married,Single,Corporate Auto,Personal Auto,...,Four-Door Car,Luxury Car,Luxury SUV,SUV,Sports Car,Two-Door Car,Offer1,Offer2,Offer3,Offer4
0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8692,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8693,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8694,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8695,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## 18.The time variable can be useful. Try to transform its data into a useful one. Hint: Day week and month as integers might be useful

In [360]:
#Already done in previous lab

In [350]:
customer_df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8697 entries, 0 to 8696
Data columns (total 23 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   state                          8697 non-null   object 
 1   customer_lifetime_value        8697 non-null   float64
 2   response                       8697 non-null   object 
 3   coverage                       8697 non-null   int64  
 4   education                      8697 non-null   int64  
 5   employmentstatus               8697 non-null   int64  
 6   gender                         8697 non-null   object 
 7   income                         8697 non-null   float64
 8   location_code                  8697 non-null   int64  
 9   marital_status                 8697 non-null   object 
 10  monthly_premium_auto           8697 non-null   int64  
 11  months_since_last_claim        8697 non-null   int64  
 12  months_since_policy_inception  8697 non-null   i

## 19.Since the model will only accept numerical data, check and make sure that every column is numerical, if some are not, change it using encoding

In [351]:
continious_scaled

,customer_lifetime_value,income
0,-0.961822,0.611236
1,0.015265,-1.234670
2,1.384454,0.364991
3,0.169690,-1.234670
4,-0.950194,0.203244
...,...,...
8692,3.822192,1.125147
8693,-0.884649,-0.526013
8694,0.289746,-1.234670
8695,0.141550,-0.514959


In [352]:
discrete

,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies
0,69,32,5,0,1
1,94,13,42,0,8
2,108,18,38,0,2
3,106,18,65,0,7
4,73,12,44,0,1
...,...,...,...,...,...
8692,73,18,89,0,2
8693,79,14,28,0,1
8694,85,9,37,3,2
8695,96,34,3,0,3


In [353]:
ordinal_encoded

,education,coverage,employmentstatus,vehicle_size,location_code
0,0,0,1,1,1
1,0,1,4,1,1
2,0,2,1,1,1
3,0,0,4,1,1
4,0,0,1,1,0
...,...,...,...,...,...
8692,0,0,1,1,2
8693,1,1,1,1,1
8694,0,1,4,1,1
8695,1,1,1,0,1


In [354]:
onehotlabels

,Arizona,California,Nevada,Oregon,Washington,Divorced,Married,Single,Corporate Auto,Personal Auto,...,Four-Door Car,Luxury Car,Luxury SUV,SUV,Sports Car,Two-Door Car,Offer1,Offer2,Offer3,Offer4
0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8692,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8693,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8694,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8695,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### As every column is numerical,we join both normalized numerical and encoded categorical columns in one dataset 

In [357]:
customer_df_cleaned_concat = pd.concat([discrete,continious_scaled,onehotlabels,ordinal_encoded,customer_df_cleaned['contracts_started_in_2011']],axis =1)

In [358]:
customer_df_cleaned_concat

,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,customer_lifetime_value,income,Arizona,California,Nevada,...,Offer1,Offer2,Offer3,Offer4,education,coverage,employmentstatus,vehicle_size,location_code,contracts_started_in_2011
0,69,32,5,0,1,-0.961822,0.611236,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0,0,1,1,1,02-24
1,94,13,42,0,8,0.015265,-1.234670,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0,1,4,1,1,01-31
2,108,18,38,0,2,1.384454,0.364991,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0,2,1,1,1,02-19
3,106,18,65,0,7,0.169690,-1.234670,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0,0,4,1,1,01-20
4,73,12,44,0,1,-0.950194,0.203244,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0,0,1,1,0,02-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8692,73,18,89,0,2,3.822192,1.125147,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0,0,1,1,2,02-10
8693,79,14,28,0,1,-0.884649,-0.526013,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1,1,1,1,1,02-12
8694,85,9,37,3,2,0.289746,-1.234670,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0,1,4,1,1,02-06
8695,96,34,3,0,3,0.141550,-0.514959,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1,1,1,0,1,02-03


In [359]:
customer_df_cleaned_concat.columns

Index(['monthly_premium_auto', 'months_since_last_claim',
       'months_since_policy_inception', 'number_of_open_complaints',
       'number_of_policies', 'customer_lifetime_value', 'income', 'Arizona',
       'California', 'Nevada', 'Oregon', 'Washington', 'Divorced', 'Married',
       'Single', 'Corporate Auto', 'Personal Auto', 'Special Auto', 'Agent',
       'Branch', 'Call Center', 'Web', 'Corporate L1', 'Corporate L2',
       'Corporate L3', 'Personal L1', 'Personal L2', 'Personal L3',
       'Special L1', 'Special L2', 'Special L3', 'Four-Door Car', 'Luxury Car',
       'Luxury SUV', 'SUV', 'Sports Car', 'Two-Door Car', 'Offer1', 'Offer2',
       'Offer3', 'Offer4', 'education', 'coverage', 'employmentstatus',
       'vehicle_size', 'location_code', 'contracts_started_in_2011'],
      dtype='object')